<a href="https://colab.research.google.com/github/DuplamenteH/DeepLearning/blob/main/Classifica%C3%A7%C3%A3o_de_roupas_Usando_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INTRODUÇÃO

## Neste projeto vamos usar um dataset Fashion MNIST que é bem padrão para estudos como este. O dataset surgiu com intuito de substituir o MNIST q é outro dataset com usado para estudos,porém bem mais fácil de desvendar,pois qualquer CNN conseguiam atingir facilmente a acurácia de 97%, logo não representava mais um desafio para visão computacional.

### *Cada imagem pertence exclusivamente a uma única classe. A tabela abaixo segue a documentação do Fashion MNIST, onde são documentados os **10** **labels** possíveis:*

<table>
  <tbody><tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</tbody></table>

<p align="center"><img src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/embedding.gif" height="300px"></p>




## Vamos usar o framework tensorflow 2 para este projetos, tanto para carregar os dados quanto para normaliza-los, e criar o modelo.


# Obtendo os dados

## Existem duas formas são elas :
* [Diretamente a partir do repositório no Github](https://github.com/zalandoresearch/fashion-mnist) .
* Carregando a partir do TensorFlow.

In [ ]:
# imports
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl

from sklearn.metrics import classification_report

# configurar a visualização
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
mpl.style.use( 'ggplot' )
plt.style.use('fivethirtyeight')
sns.set(context="notebook", palette="dark", style = 'whitegrid' , color_codes=True)

## Carregando os Dados nas variaeis

In [ ]:
# carregando os dados do Fashion MNIST
(X_train_orig, y_train_orig),(X_test_orig, y_test_orig) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# Labels de acordo com a documentação ,obs:Estou traduzindo as mesmas.
class_name = [ 'Camisetas/Top','Calça', 'Suéter',
              'Vestidos', 'Casaco', 'Sandálias',
              'Camisas', 'Tênis', 'Bolsas', 'Botas']

# Exploração dos Dados

### Agora vamos começar a entender nossos dados, como baixamos nossas dados da API do tensorflow não precisamos nos preocupar com dados nulos e esses demais erros.
### Para começar vou verificar as dimensões dos nossos arrays q devem ser de 60.000 amostras de treino e 10.000 de teste, sendo cada imagem com 28x28 pixels com apenas com tons de cinza, ou seja , tendo apenas 1 canal.

In [ ]:
# ver a dimensionalidade dos DataFrames
print("Dimensionalidade dos DataFrames:")
print("X_train_orig:", X_train_orig.shape)
print("y_train_orig:", y_train_orig.shape)
print("X_test_orig:", X_test_orig.shape)
print("y_test_orig:", y_test_orig.shape)


Agora vamos vericar se os datasets (treino e teste) estão com uma proporção idel entre as classes

In [ ]:
# verificando os val únicos por classes(treino)
print("y_train_orig:")
np.unique(y_train_orig, return_counts=True)

Por último, vamos visualizar algumas imagens com seus respectivos labels

In [ ]:
plt.figure(figsize=(8,8))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_orig[i], cmap=plt.cm.binary)
    plt.xlabel(class_name[y_train_orig[i]])
plt.tight_layout()

# Pré-Processamentos dos Dados

Antes de criar o modelo e treinar a rede, vamos noramalizar nossas imagens.

## Normalizar os pixels
Para normalizar dos pixels, basta dividi-lo pelo valor maximo que um pixel pode atingir , então vamos digidir as imagens por 255.0

In [ ]:
"""
    Aqui vamos usar uma função lambda para percorrer 
    nossas imagens e dividir todas por 255.0
"""
f = lambda x :(x/255.0).astype("float32")

X_train = f(X_train_orig)
X_test = f(X_test_orig)

## Redimensionar as Imagens

Um dos principais problemas ao se trabalhar com dados não-estruturados, é a dimensão desde dados, já que o primeiro layer da nossa CNN espera um único tensor que contenha todos os pixels da imagem, pois o TensorFlow espera uma lista com 4 dim, e não 60000 itens com 28 x 28 x 1.

In [ ]:
# redimensionar as imagens
X_train = X_train.reshape((X_train.shape[0],28,28,1))
X_test = X_test.reshape((X_test.shape[0],28,28,1))


print("X_train: {}".format(X_train.shape))
print("X_test:\t{}".format(X_test.shape))

## One-Hot Encoding
Vamos utilizamos para otimizar nossos ganhos com o modelo, pois se não aplicarmos teremos que usar `loss="sparse_categorical_crossentropy"`.Apenas usar essa loss não é problema o problema em que o modelo de deep learning vai da a entender que a existe uma relação entre a ordem dos labels e isso será muito prejudical ao nosso trabalho.

In [ ]:
y_train = keras.utils.to_categorical(y_train_orig)
y_test = keras.utils.to_categorical(y_test_orig)

# Criando nossas Rede Neural.

Existem  ***N*** formar de se contruir uma CNN. Nós vamos utilizar um arquiterua simplificada da VGGNET, que é implementtada em um artigo do ***Adrian Rosebroke dono do site pyimagesearch e também foi modificada por Carlos Melo dono do blog sigmoidal.***



***CONV => RELU => CONV=> RELU => POOL***

In [ ]:
def buildModel():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, 3, padding="same", activation='relu',))
    model.add(keras.layers.BatchNormalization(axis=1))
    model.add(keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu'))
    model.add(keras.layers.BatchNormalization(axis=1))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))

    # second CONV => RELU => CONV => RELU => POOL layer set
    model.add(keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(keras.layers.BatchNormalization(axis=1))
    model.add(keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(keras.layers.BatchNormalization(axis=1))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))

    # first (and only) set of FC => RELU layers
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.5))

    # softmax classifier
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model
model = buildModel()

Como visto acima, nossos dados estão com os labels one-hot encoded, será utilizado `loss="categorical_crossentropy"` para compilar nosso modelo.

Vamos também informar que queremos 1/3 de nossos dados para validação

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])


Agora vamos treinar o modelo e salvar suas informações do treinamento

continuar....

In [ ]:
history = model.fit(X_train, y_train,epochs=40, validation_split= 0.3)

# Avaliando o Modelo
Depois de treinado  nosso modelo atingiu uma acurácia de A nos dados de treino e A nos dados de validação.
Porém, se obervamos melhor veremos q ela não teve um desempenho bom para categoria " Camisas".
Para aumentar essa acuracia podemos usar tecnicas de data augmentation.

In [ ]:
y_hat = model.predict(X_test)
y_hat_class = np.argmax(y_hat, axis=1)
print(classification_report(y_test_orig, y_hat_class, target_names=class_name))

Agora vamos observar o grafico do nosso treino.

In [ ]:
pd.DataFrame(history.history).plot()
plt.show()

Olhando o grafico podemos identicar um pequeno overfitting, mas que n prejudicou tanto o desempenho da nossa CNN perante os dados de validação.

Agora vamos Colocar nossas dados de teste para ver quão bom nosso modelo está.

In [ ]:
score = model.evaluate(X_test, y_test)

# verificar o desempenho do modelo
print('Loss: {:.4f}'.format(score[0]))
print('Acurácia: {:.4f}'.format(score[1]))

Como podemos ver acima nosso modelo esta muito bom, ou seja , ele permanece genérico o suficiente para lidar com dados novos.

# Melhorias

Para melhorar ainda o desempenho, devemos fazer um data augmentation, no nosso dataset de treino, com isso iremos melhorar a acuracia na detecção de "Camisas" e "Camisetas/Top".
-=-----=---=---=---
Podemos também mudar o otmizador , e ver se muda algo em nossa acuracia